# LAV BEV Training on Colab

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/MichaelBosello/carla-colab/blob/master/carla-simulator.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

---
---

## Introduction
[Carla](https://github.com/carla-simulator/carla) is an open-source simulator for autonomous driving research.

Since it is hardware demanding, it could be useful for students to run it on colab.

---

## Acknowledgements

Thanks to *Tomohiro*  for developing the code that allows remote access to colab.
Check out his [page](https://internet-of-tomohiro.netlify.app/google_colab/vnc.en.html) and [GitHub](https://github.com/demotomohiro/remocolab).



## Requirements

*   Ssh client (already present in all recent os).


## Installation 

4. Wait the end of the process
  * It sets up OpenSSH server, VirtualGL and TurboVNC.
  * Then, access passwords are shown



## Download Jekyll's LAV

Clone myown LAV from(https://github.com/jekyll88/lav)


In [2]:
! git clone https://github.com/jekyll88/lav.git


Cloning into 'lav'...
remote: Enumerating objects: 1958, done.
remote: Counting objects: 100% (1958/1958), done.
remote: Compressing objects: 100% (1824/1824), done.
remote: Total 1958 (delta 123), reused 1955 (delta 123), pack-reused 0
Receiving objects: 100% (1958/1958), 6.52 MiB | 6.63 MiB/s, done.
Resolving deltas: 100% (123/123), done.


## Install necessary packages

In [5]:
#! sudo -u colab mkdir /home/colab/carla
#! sudo -u colab tar -xf /home/colab/CARLA.tar.gz -C /home/colab/carla
#%cd lav
!pip3 install einops
import sys
sys.path.append('/content/lav')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Preparing Data


In [6]:
import tqdm
import torch
from lav.lav_privileged import LAV
from lav.utils.datasets import get_data_loader
from lav.utils.logger import Logger


ModuleNotFoundError: ignored

## Training

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import some useful packages for this homework


# !nvidia-smi # list your current GPU
